In [1]:
import pandas as pd
import re
 
stage_1 = pd.read_csv('../../cleaning_stages/1_openrefine/cddb-tsv.tsv', sep='\t')
stage_5 = pd.read_csv('../../cleaning_stages/5_genre_repair/cddb_4.tsv', sep='\t')
final_drop = pd.read_csv('../../cleaning_stages/6_openrefine_drop_bad/cddb-4-drop.tsv', sep='\t')

## IC 1: Null Year

In [2]:
def null_years(df):
    return df[df['year'].isna()]

def null_years_percent(df):
    return len(null_years(df)) / len(df)

In [3]:
null_years_stage_1 = null_years_percent(stage_1)
null_years_stage_5 = null_years_percent(stage_5)

print(f'stage 1 percent of null years: {round(null_years_stage_1, 2)}')
print(f'stage 5 percent of null years: {round(null_years_stage_5, 2)}')
print(f'Difference: {round(null_years_stage_1 - null_years_stage_5, 2)}')

stage 1 percent of null years: 0.46
stage 5 percent of null years: 0.21
Difference: 0.25


## IC 2: Null Genre

In [4]:
def null_genre(df):
    return df[df['genre'].isna()]

def null_genre_percent(df):
    return len(null_genre(df)) / len(df)

In [5]:
null_genre_stage_1 = null_genre_percent(stage_1)
null_genre_stage_5 = null_genre_percent(stage_5)

print(f'stage 1 percent of null genre: {round(null_genre_stage_1, 2)}')
print(f'stage 5 percent of null genre: {round(null_genre_stage_5, 2)}')
print(f'Difference: {round(null_genre_stage_1 - null_genre_stage_5, 2)}')

stage 1 percent of null genre: 0.35
stage 5 percent of null genre: 0.0
Difference: 0.35


## IC 3: Years with 9999 

In [6]:
def exists_9999_years(df):
    return df[df['year'] == '9999']

In [7]:
print(exists_9999_years(final_drop))

Empty DataFrame
Columns: [artist, category, genre, title, tracks, year, id, merged_values]
Index: []


## Dropping Rows

For our final openrefine we used that to drop rows that could not be repaired.

We ended up having to drop about 22% of the data, which is significantly better than dropping 50% like we would have had to do before cleaning. 

In [8]:
print(f'Stage 5 Count: {len(stage_5)}')
print(f'Final Count: {len(final_drop)}')
print(f'Number of Nulls Dropped: {len(stage_5) - len(final_drop)}')
print(f'Loss Percent: {round(1.0 - (len(final_drop) / len(stage_5)), 2) * 100}%')

Stage 5 Count: 9763
Final Count: 7663
Number of Nulls Dropped: 2100
Loss Percent: 22.0%


In [9]:
integrity_constraints = [null_years, null_genre, exists_9999_years]

def run_tests(df):
    for ic in integrity_constraints:
        result = "PASS" if ic(df).empty else "FAIL"
        print(f'{ic.__name__} .... {result}')

In [10]:
run_tests(final_drop)

null_years .... PASS
null_genre .... PASS
exists_9999_years .... PASS


## Additional IC Constraints

In [11]:
stage_7 = pd.read_csv('../../cleaning_stages/7_unit_code_repair/cddb_5.tsv', sep='\t')
stage_7_tracks = pd.read_csv('../../cleaning_stages/7_unit_code_repair/cddb_tracks_2.tsv', sep='\t')
stage_2_tracks = pd.read_csv('../../cleaning_stages/2_tracks_split/cddb_tracks.tsv', sep='\t')

# comparing stage_7 to stage_1
# comparing stage_7_tracks to stage_2_tracks

## IC 4: duplicate ID check

In [12]:
def duplicate_id_Check(df):
    ic4_violations = df[df.groupby(['id'])['id'].transform('count') > 1]
    return ic4_violations

In [13]:
id_check_stage_1 = len(duplicate_id_Check(stage_1))
id_check_stage_7 = len(duplicate_id_Check(stage_7))
diff = id_check_stage_1 - id_check_stage_7
Success = "Pass" if diff == 0 & id_check_stage_7 <= id_check_stage_1 else "Fail"


print(f'stage 1 a total of: {id_check_stage_1} duplicate id(s)')
print(f'stage 7 a total of: {id_check_stage_7} duplicate id(s)')
print(f'Difference: {diff}')
print(f"Success: {Success}")


stage 1 a total of: 0 duplicate id(s)
stage 7 a total of: 0 duplicate id(s)
Difference: 0
Success: Pass


## IC 5: Mangled Unit Code Check

In [14]:
def check_mangled_code(string):
    check_string = string.lower().strip()
    reg = "[^a-zA-Z0-9_][^a-zA-Z0-9_][^a-zA-Z0-9_][^a-zA-Z0-9_]"
    p = re.compile(reg)

    # check if ?????
    # two ?? in a row is mangled code
    if "??" in check_string:
        return True
    
    # check if mangled
    else:
        return bool(p.match(check_string))

In [15]:
def mangled_unit_code_Check(df):
    violations = 0
    
    for i, row in df.iterrows():
        # 1. get row values as string
        row_str = df.iloc[i,:].to_string(header=False, index=False)
        row_str = row_str.replace(" ", "").replace('\n', ',').replace('\r', ',').replace('\t', '').strip()
        
        # 2. condition checking
        if check_mangled_code(row_str) == True:
            # 3. increment violations
            violations += 1
        else:
            pass

    return violations / len(df)

In [16]:
unit_check_stage_1 = mangled_unit_code_Check(stage_1)
unit_check_stage_7 = mangled_unit_code_Check(stage_7)
unit_diff = unit_check_stage_1 - unit_check_stage_7
unit_Success = "Pass" if unit_check_stage_7 <= unit_check_stage_7 else "Fail"


print(f'stage 1 a total of: {round(unit_check_stage_1,2)}% mangled unit code violations')
print(f'stage 7 a total of: {round(unit_check_stage_7,2)}% mangled unit code violations')
print(f'Difference: {round(unit_diff,2)}')
print(f"Success: {unit_Success}")

stage 1 a total of: 0.03% mangled unit code violations
stage 7 a total of: 0.0% mangled unit code violations
Difference: 0.03
Success: Pass


In [17]:
unit_check_stage_2_t = mangled_unit_code_Check(stage_2_tracks)
unit_check_stage_7_t = mangled_unit_code_Check(stage_7_tracks)
unit_diff_t = unit_check_stage_2_t - unit_check_stage_7_t
unit_Success_t = "Pass" if unit_check_stage_7_t <= unit_check_stage_2_t else "Fail"


print(f'stage 2 tracks a total of: {round(unit_check_stage_2_t,2)}% mangled unit code violations')
print(f'stage 7 tracks a total of: {round(unit_check_stage_7_t,2)}% mangled unit code violations')
print(f'Difference: {round(unit_diff_t,2)}')
print(f"Success: {unit_Success_t}")

stage 2 tracks a total of: 0.0% mangled unit code violations
stage 7 tracks a total of: 0.0% mangled unit code violations
Difference: 0.0
Success: Pass


## IC 6: Every Album Id in cddb has at least one song in tracks

In [18]:
# need a stage 2 file for this since it is post split
stage_2 = pd.read_csv('../../cleaning_stages/2_tracks_split/cddb_1.tsv', sep='\t')

In [19]:
def album_dependancy_check(df, tracks):
    violations = 0

    for i, row in df.iterrows():
        album_id = row[4]

        tracks_for_album = tracks[tracks['Album Id'] == album_id]
        if len(tracks_for_album) == 0:
            violations += 1

    return violations / len(df)

In [23]:
album_check_stage_2 = round(album_dependancy_check(stage_2, stage_2_tracks),2)
album_check_stage_7 = round(album_dependancy_check(stage_7, stage_7_tracks),2)
album_diff = album_check_stage_2 - album_check_stage_7
album_Success = "Pass" if album_check_stage_7 <= album_check_stage_2 else "Fail"


print(f'stage 2 tracks a total of: {album_check_stage_2}% album dependency violations')
print(f'stage 7 tracks a total of: {album_check_stage_7}% album dependency violations')
print(f'Difference: {round(album_diff,2)}')
print(f"Success: {album_Success}")

stage 2 tracks a total of: 0.0% album dependency violations
stage 7 tracks a total of: 0.0% album dependency violations
Difference: 0.0
Success: Pass
